# Surprise

En esta practica trabajaremos con el paquete [Surprise](https://surprise.readthedocs.io/en/stable/index.html) el cual se trata de un paquete basado en sistemas recomendadores, por lo que estará formado por diversos algoritmos y conjuntos de datos para aplicarlos en este fin.

En este caso el paquete no se encuentra instalado en Google Colab por lo que la primera tarea necesaria es instalarlo.

In [1]:
!pip install surprise

In [2]:
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split

# Parte I

En esta parte ejecutaremos un dataset de un rankings creado por nosotros mismos, y evaluaremos con un algoritmo recomendador determinado para ver su precisión mediante la validación cruzada.

In [3]:
# Importamos los paquetes y liberías necesarios

import pandas as pd

from surprise import Reader
from surprise import accuracy
from surprise import NormalPredictor
from surprise.model_selection import cross_validate


In [4]:
# Creamos el data frame (destacar que el nombre de las columnas es irrelevante)
ratingsDict = {'itemID': [1, 1, 1, 2, 2],
                'userID': [9, 32, 2, 45, 23],
                'rating': [3, 2, 4, 3, 1]}
df = pd.DataFrame(ratingsDict)
df.head()

,itemID,userID,rating
0,1,9,3
1,1,32,2
2,1,2,4
3,2,45,3
4,2,23,1


In [5]:
# Una variable necesaria para pasar los datos al formato de la librería Surprise es reader de la escala de los datos
reader = Reader(rating_scale=(1, 5))

# Creamos los datos dando la importancia a que tienen que tener el orde de user ID, item ID y el ranking
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)

En este caso el algoritmo que usaremos es un [predictor normal](https://surprise.readthedocs.io/en/stable/basic_algorithms.html) que basa su predicción en un ranking aleatorio de la distribución del conjunto de entrenamiento.

In [6]:
recommender = NormalPredictor()

In [7]:
# Dividimos en train y test
trainset, testset = train_test_split(data, test_size=.25)

# Entrenamos y realizamos predicciones
recommender.fit(trainset)
predictions = recommender.test(testset)

# Calculamos el error.
accuracy.rmse(predictions)

RMSE: 1.5766


1.5766049445782053

# Parte II

In [8]:
# Importamos los paquetes y librerías necesarios

from surprise import SVD

Cargamos y mostramos los datos, en este caso se trata de una colección de datos obtenida de la página MovieLens en 1997 y está formado por 4 categorías principales, el ID del usuario, el ID de la película, el ranking asignado y una marca de tiempo (características las cuales corresponden a cada columna mostrada a continuación). 

In [9]:
# Cargamos el conjunto de datos movielens-100k
data = Dataset.load_builtin('ml-100k')

# user id | item id | rating | timestamp
data.raw_ratings[:10]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

En este caso ejecutaremos el algoritmo [SVD](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD) que está basado en la factorización de matrices el cual ganó popularidad en el [concurso de Netflix](https://www.netflixprize.com/) sobre sistemas recomendadores.

In [10]:
# Creamos la variable con nuestro recomendador SVD
recommender = SVD()

El siguiente paso que realizaremos es aplicar la validación cruzada para ver el resultado que da evaluándola con dos métricas especificas (RMSE - Root Mean Squared Error y MAE - Mean Absolute Error).

In [11]:
# Aplicamos la validadación cruzada pasandole el algoritmo recomendador, los datos y el valor para dividir los datos
cross_validation = cross_validate(recommender, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
print(cross_validation)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9370  0.9303  0.9370  0.9432  0.9330  0.9361  0.0044  
MAE (testset)     0.7367  0.7346  0.7385  0.7414  0.7386  0.7379  0.0022  
Fit time          5.67    5.68    5.67    5.68    5.66    5.67    0.01    
Test time         0.15    0.22    0.22    0.15    0.21    0.19    0.04    
{'test_rmse': array([0.93701361, 0.93030946, 0.9370251 , 0.94316901, 0.93297642]), 'test_mae': array([0.73665448, 0.73463423, 0.73850309, 0.74138169, 0.73856209]), 'fit_time': (5.6687586307525635, 5.68136739730835, 5.671870231628418, 5.68377161026001, 5.664249420166016), 'test_time': (0.14521574974060059, 0.21622061729431152, 0.22072720527648926, 0.145432710647583, 0.21316981315612793)}


Además podemos predecir la valoración de un usuario sobre un correspondiente elemento:

In [12]:
# Identificador del usuario
uId = str(196)

# Identificador de la película
iId = str(302)

pred = recommender.predict(uId, iId, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.25   {'was_impossible': False}
